In [2]:
import os
import json
import pandas as pd 
import traceback

In [5]:
from langchain.chat_models import ChatOpenAI


In [6]:
from dotenv import load_dotenv
load_dotenv()


True

In [9]:
KEY = os.getenv('OPENAI_API_KEY')

In [10]:
KEY

'sk-proj-5tfVRTX5YpSUjid2qzeKT3BlbkFJWsvdi2lNHoSAQLXbngv1'

In [13]:
llm = ChatOpenAI(openai_api_key=KEY, model_name= 'gpt-3.5-turbo', temperature=0.5)


In [14]:
llm

ChatOpenAI(cache=None, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, client=APIRemovedInV1Proxy, model_name='gpt-3.5-turbo', temperature=0.5, model_kwargs={}, openai_api_key='sk-proj-5tfVRTX5YpSUjid2qzeKT3BlbkFJWsvdi2lNHoSAQLXbngv1', openai_api_base='', openai_organization='', openai_proxy='', request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=None, tiktoken_model_name=None)

In [15]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2


In [17]:
RESPONSE_JSON = {
    '1' : {
        'mcq': 'multiple choice question',
        'options':{
            'a': 'choice here',
            'b': 'choice here',
            'c': 'choice here',
            'd': 'choice here'
        },
        'correct': 'correct answer'
            },
    '2' : {
        'mcq': 'multiple choice question',
        'options':{
            'a': 'choice here',
            'b': 'choice here',
            'c': 'choice here',
            'd': 'choice here'
        },
        'correct': 'correct answer'
            },
    '3' : {
        'mcq': 'multiple choice question',
        'options':{
            'a': 'choice here',
            'b': 'choice here',
            'c': 'choice here',
            'd': 'choice here'
        },
        'correct': 'correct answer'
            },
}

In [16]:
TEMPLATE = """ 
Text: {text}
You are an expert MCQ maker. Giiven the above text, it is \
your job to create a quiz of {number} MCQs for {subject} students \
in {tone} tone.
Make sure the questions are not repeated and check all the questions\
to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and \
use it as a guide. Ensure to make {number} MCQs.
# RESPONSE_JSON
{response_json}

"""

In [18]:
quiz_generation_prompt = PromptTemplate(
    input_variables=['text','number','subject','tone','response_json'],
    template= TEMPLATE
)

In [19]:
quiz_chain =  LLMChain(llm=llm,prompt=quiz_generation_prompt, output_key='quiz', verbose= True)


In [20]:
TEMPLATE2 = """ 
You are an expert english grammarian and writer. Given a Multiple \
Choice Question Quiz for {subject} students.
You need to evaluate the complexity of the questions and give a \
complete analysis of the quiz. Only use at max 50 words for \
complexity analysis. if the quiz is not at par with the cognitive\
and analytical abilities of the students, update the quiz questions\
which needs to be changed and change the tone such that it \
perfectly fits the students abbilities.
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [69]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=['subject', 'quiz'],
    template=TEMPLATE2
)

In [22]:
review_chain = LLMChain(llm = llm, prompt=quiz_evaluation_prompt,output_key='review', verbose=True)


In [57]:
generate_evaluate_chain = SequentialChain(
    chains=[quiz_chain, review_chain],
    input_variables=['text','number','subject','tone','response_json'],
    output_variables=['quiz','review'])

In [58]:
file_path = '/Users/aws/Documents/projectmcq/data.txt'

In [59]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [60]:
print(TEXT)

Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2][3]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[4][5] When applied to business problems, it is known under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.

The mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis (EDA) through unsupervised learning.[7][8]

From a theoreti

In [68]:
#converting python dict into a json formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [62]:
NUMBER = 5
SUBJECT = 'machine learning'
TONE = 'simple'

In [67]:
# setting up token usgae tracking in langchain
with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
            'text': TEXT,
            'number': NUMBER,
            'subject' : SUBJECT,
            'tone' : TONE,
            'response_json' : json.dumps(RESPONSE_JSON)
        }
           )



> Entering new  chain...
Prompt after formatting:
 
Text: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2][3]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[4][5] When applied to business problems, it is known under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.

The mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis

AttributeError: module 'openai' has no attribute 'error'